In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import swifter

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# load environment variables from .env
load_dotenv(find_dotenv())

# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

dir_raw = data_dir/'raw'
dir_test = data_dir/'test'
dir_int = data_dir/'interim'

gdb_int = dir_int/'interim.gdb'

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import ba_tools

In [3]:
raw_trips_tbl = dir_raw/'raw_trips.csv'

In [23]:
raw_trips_df = pd.read_csv(raw_trips_tbl, index_col=0)

raw_trips_df.head()

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum
0.0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145
0.0,8.260506,12.466667,"{'x': -123.150157, 'y': 44.933511, 'spatialRef...",244210043
1.0,2.013863,8.750000,"{'x': -123.312642, 'y': 44.929472, 'spatialRef...",244210043
2.0,0.547428,7.516667,"{'x': -123.30801, 'y': 44.9299, 'spatialRefere...",244210043
3.0,0.133595,3.100000,"{'x': -123.315767, 'y': 44.922983, 'spatialRef...",244210043


In [24]:
raw_trips_df.SHAPE = raw_trips_df.SHAPE.apply(lambda val: eval(val))

In [25]:
raw_trips_df['coord_x'] = raw_trips_df.SHAPE.apply(lambda val: val['x'])
raw_trips_df['coord_y'] = raw_trips_df.SHAPE.apply(lambda val: val['y'])

In [29]:
out_df = raw_trips_df[['travel_distance_miles', 'travel_time_minutes', 'store_locnum', 'coord_x', 'coord_y']].reset_index()
out_df.head()

,index,travel_distance_miles,travel_time_minutes,store_locnum,coord_x,coord_y
0,0.0,13.261921,25.033333,413963145,-123.026415,44.877280
1,0.0,8.260506,12.466667,244210043,-123.150157,44.933511
2,1.0,2.013863,8.750000,244210043,-123.312642,44.929472
3,2.0,0.547428,7.516667,244210043,-123.308010,44.929900
4,3.0,0.133595,3.100000,244210043,-123.315767,44.922983


In [30]:
out_df.to_csv(dir_raw/'trips.csv')